In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import h5py
import math as ma

In [ ]:
df = pd.read_csv("/Users/eormacstudio/Documents/GitHub/GRANDWin/data/raw/observation_id/observation_id.csv", header=0, engine='python')
df['date'] = df.starttime_utc.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z").date())
df['date_time'] = df.starttime_utc.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z"))
df['partition'] = pd.factorize(df['date'])[0] + 1
df = df[df['partition'] == 1].reset_index(drop=True)

---

#### Checking the results normal vs faster algorithm

In [ ]:
df_sel_rfi_fast = pd.DataFrame()
transform_tb = np.repeat(np.arange(0,14,1), 4)

for n in range(3):
    with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/faster/win_z_scores_data_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
        data = f["wins_z_score"][:]
        obs_id = f["obs_id"][:]
        time_blocks = f["time_blocks"][:]

    flat_data = data.reshape(-1, data.shape[-1])
    
    # Create DataFrame
    df_temp = pd.DataFrame(flat_data, columns=["XX", "XY", "YX", "YY"])

    df_temp["obs_id"] = np.tile(obs_id, data.shape[1]*data.shape[2])
    df_temp["time_index"] = np.repeat(np.arange(data.shape[0]), data.shape[1] * data.shape[2])
    df_temp["frequency"] = np.tile(np.arange(data.shape[2]), data.shape[0] * data.shape[1])
    df_temp["antenna"] = np.tile(np.repeat(np.arange(data.shape[1]), data.shape[2]), data.shape[0])
    df_temp["obs_id"] = df_temp['time_index'].map(lambda t: obs_id[t])
    df_temp["timeblock"] = df_temp['time_index'].map(lambda t: int(time_blocks[t]))
    #df_temp['timeblock'] = df_temp.timeblock.apply(lambda x: np.where(transform_tb == x)[0].tolist())
    #df_temp = df_temp.explode('timeblock').reset_index(drop=True)

    df_sel_rfi_fast = pd.concat([df_sel_rfi_fast, df_temp]).reset_index(drop = True)


In [ ]:
df_sel_rfi_slow = pd.DataFrame()
for o in range(3):
    aa = pd.read_csv("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/slower/both_side_selected_outliers_3_sigma_day_1_grid_[%s]_iter_100_real_part_8s.csv" %(o), header=None)
    aa.columns = ['xx', 'yy', 'obs_id', 'timeblocks', 'tile', 'freqs', 'z_score', 'polarization']
    aa = aa[~aa['z_score'].isna()].reset_index(drop=True)
    aa['z_score'] = aa['z_score'].astype(float)
    aa['floor_z_score'] = aa['z_score'].apply(lambda x: ma.floor(x))
    df_sel_rfi_slow = pd.concat([df_sel_rfi_slow, aa]).reset_index(drop = True)

In [ ]:
np.unique(df_sel_rfi_fast['obs_id'])

In [ ]:
df_sel_rfi_fast[df_sel_rfi_fast['obs_id'] == 1095449728].head(10)

In [ ]:
df_sel_rfi_slow[(df_sel_rfi_slow['obs_id'] == 1095449728) & (df_sel_rfi_slow['freqs'] == 0) & (df_sel_rfi_slow['tile'] == 0) & (df_sel_rfi_slow['timeblocks'] == 0)]

---

#### Min max comparison

In [ ]:
df_stat_slow = pd.DataFrame()
for o in range(3):
    oo = pd.read_csv("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/slower/both_side_outliers_statistics_3_sigma_day_1_grid_[%s]_iter_100_real_part_8s.csv" %(o), header=None)
    oo.columns = ['tile', 'freqs', 'polarization', 'skewness', 'max', 'min', 'mean', 'win_mean', 'std', 'win_std', 'gamma_final', 'number_of_outliers']
    oo['grid'] = o
    df_stat_slow = pd.concat([df_stat_slow, oo]).reset_index(drop = True)

In [ ]:
df_stat_fast = pd.DataFrame()
for o in range(3):
    oo = pd.read_parquet("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/faster/outlier_statistics_day_1_grid_%s_integration_8_real.parquet" %(o))
    oo['grid'] = o
    df_stat_fast = pd.concat([df_stat_fast, oo]).reset_index(drop = True)

In [ ]:
df_gamma_fast = pd.DataFrame()
for o in range(3):
    oo = pd.read_parquet("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/faster/final_gamma_day_1_grid_%s_integration_8_real.parquet" %(o))
    oo['grid'] = o
    df_gamma_fast = pd.concat([df_gamma_fast, oo]).reset_index(drop = True)

In [ ]:
df_stat_slow[(df_stat_slow['tile'] == 0) & (df_stat_slow['freqs'] == 8)]

In [ ]:
df_stat_fast[(df_stat_fast['antenna'] == 0) & (df_stat_fast['frequency'] == 8) & (df_stat_fast['polarization'].isin([0,3]))]

In [ ]:
df_gamma_fast[(df_gamma_fast['antenna'] == 0) & (df_gamma_fast['frequency'] == 7) & (df_stat_fast['polarization'].isin([0,3]))]